In [ ]:
# ⬅︎ Cell 1: Imports & Config
import os
import json
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
# מודלים
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# Optuna
import optuna
from optuna.pruners import MedianPruner

warnings.filterwarnings("ignore")

RANDOM_STATE = 42
DATA_PATH = "../data/raw/telco_churn.csv"  # התאם אם צריך
TARGET_COL = "Churn"


In [2]:
# ⬅︎ Cell 2: Load & Basic cleaning
df = pd.read_csv(DATA_PATH)

# Telco quirks
# TotalCharges מגיע כמחרוזת; ננקה לרמה נומרית ונטפל בחסרים
df["TotalCharges_num"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
mask_na = df["TotalCharges_num"].isna()
df.loc[mask_na, "TotalCharges_num"] = df.loc[mask_na, "MonthlyCharges"] * df.loc[mask_na, "tenure"].clip(lower=1)

# יעד לבוליאני
df[TARGET_COL] = df[TARGET_COL].map({"Yes": 1, "No": 0}).astype(int)

# פיצ’ר־אנג’ינירינג קל בתוך מחלקת טרנספורמר כדי לשלב ב-Pipeline
service_cols = [
    "PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup",
    "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"
]

class AddFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, service_cols):
        self.service_cols = service_cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        # תקופת מנוי בשנים
        X["tenure_years"] = X["tenure"] / 12.0
        
        # יחס חיובים לותק (charge per month בפועל)
        X["charges_per_tenure"] = X["TotalCharges_num"] / np.maximum(X["tenure"], 1)
        
        # ספירת שירותים פעילים (Yes)
        def yes(x): 
            return 1 if str(x).strip().lower()=="yes" else 0
        X["services_count"] = X[self.service_cols].applymap(yes).sum(axis=1)
        
        # אינדיקטורים שימושיים
        X["has_streaming"] = ((X.get("StreamingTV","No")=="Yes") | (X.get("StreamingMovies","No")=="Yes")).astype(int)
        X["has_fiber"] = (X.get("InternetService","None")=="Fiber optic").astype(int)
        X["is_electronic_check"] = (X.get("PaymentMethod","").str.contains("Electronic check", case=False, na=False)).astype(int)
        X["auto_pay"] = (X.get("PaymentMethod","").str.contains("automatic", case=False, na=False)).astype(int)
        
        # מיפוי משך חוזה לחודשים
        contract_map = {"Month-to-month": 1, "One year": 12, "Two year": 24}
        X["contract_term"] = X.get("Contract","Month-to-month").map(contract_map).fillna(1).astype(int)
        
        # אינטראקציות פשוטות
        X["monthly_x_term"] = X["MonthlyCharges"] * X["contract_term"]
        X["tenure_x_services"] = X["tenure"] * X["services_count"]
        
        return X


In [3]:
# ⬅︎ Cell 3: Split
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RANDOM_STATE
)

pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / max(pos, 1)
scale_pos_weight


2.7691131498470947

In [4]:
# ⬅︎ Cell 4: Preprocessing pipeline
feature_adder = AddFeatures(service_cols=service_cols)

numeric_proc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))  # with_mean=False לשילוב עם sparse
])

categorical_proc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_proc, selector(dtype_include=["int64","float64","int32","float32","uint8","bool"])),
        ("cat", categorical_proc, selector(dtype_include=object))
    ],
    remainder="drop"
)

# נבנה Pipeline כללי: add features -> preprocess -> model (יוחדר בדינמיות)
def make_pipeline(model):
    return Pipeline(steps=[
        ("add", feature_adder),
        ("prep", preprocessor),
        ("model", model)
    ])


In [5]:
# ⬅︎ Cell 5: Eval helper
def evaluate(model, X_tr, y_tr, X_va, y_va):
    proba_tr = model.predict_proba(X_tr)[:,1]
    proba_va = model.predict_proba(X_va)[:,1]
    metrics = {
        "roc_auc_train": roc_auc_score(y_tr, proba_tr),
        "pr_auc_train":  average_precision_score(y_tr, proba_tr),
        "roc_auc_valid": roc_auc_score(y_va, proba_va),
        "pr_auc_valid":  average_precision_score(y_va, proba_va),
    }
    return metrics


In [6]:
# ⬅︎ Cell 6: Optuna objective (5-fold CV on train)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def objective(trial):
    model_type = trial.suggest_categorical("model_type", ["xgb", "lgbm"])
    
    if model_type == "xgb":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 200, 800),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 8),
            "learning_rate": trial.suggest_float("xgb_lr", 1e-2, 3e-1, log=True),
            "subsample": trial.suggest_float("xgb_subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample", 0.6, 1.0),
            "reg_alpha": trial.suggest_float("xgb_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("xgb_lambda", 1e-8, 10.0, log=True),
            "min_child_weight": trial.suggest_int("xgb_min_child_weight", 1, 10),
            "gamma": trial.suggest_float("xgb_gamma", 0.0, 5.0),
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "objective": "binary:logistic",
            "tree_method": "hist",
            "eval_metric": "aucpr",
            "scale_pos_weight": scale_pos_weight
        }
        base_model = XGBClassifier(**params)
    else:
        params = {
            "n_estimators": trial.suggest_int("lgbm_n_estimators", 300, 1000),
            "num_leaves": trial.suggest_int("lgbm_num_leaves", 31, 255),
            "learning_rate": trial.suggest_float("lgbm_lr", 1e-2, 3e-1, log=True),
            "subsample": trial.suggest_float("lgbm_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("lgbm_colsample", 0.5, 1.0),
            "min_child_samples": trial.suggest_int("lgbm_min_child_samples", 5, 100),
            "reg_alpha": trial.suggest_float("lgbm_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("lgbm_lambda", 1e-8, 10.0, log=True),
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "objective": "binary",
            "metric": "average_precision",
            "verbose": -1,
            "scale_pos_weight": scale_pos_weight
        }
        base_model = LGBMClassifier(**params)
    
    pipe = make_pipeline(base_model)

    pr_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(cv.split(X_train, y_train), start=1):
        X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        y_tr, y_va = y_train[tr_idx], y_train[va_idx]

        pipe.fit(X_tr, y_tr)
        proba = pipe.predict_proba(X_va)[:, 1]
        pr_auc = average_precision_score(y_va, proba)
        pr_aucs.append(pr_auc)
        
        trial.report(np.mean(pr_aucs), step=fold)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return float(np.mean(pr_aucs))


In [7]:
# ⬅︎ Cell 7: Run study
study = optuna.create_study(direction="maximize", pruner=MedianPruner())
study.optimize(objective, n_trials=40, timeout=None, n_jobs=1, show_progress_bar=True)

print("Best value (PR-AUC):", study.best_value)
print("Best params:", study.best_params)


[I 2025-08-30 20:01:44,262] A new study created in memory with name: no-name-47eb74f8-920a-4f33-ac8e-8a16b61f3572
Best trial: 0. Best value: 0.58022:   2%|▎         | 1/40 [00:07<04:51,  7.47s/it]

[I 2025-08-30 20:01:51,733] Trial 0 finished with value: 0.5802196151770449 and parameters: {'model_type': 'lgbm', 'lgbm_n_estimators': 899, 'lgbm_num_leaves': 175, 'lgbm_lr': 0.16126147911448108, 'lgbm_subsample': 0.9292528655795912, 'lgbm_colsample': 0.8620782818264461, 'lgbm_min_child_samples': 50, 'lgbm_alpha': 9.392959240742093e-05, 'lgbm_lambda': 0.00406647984315883}. Best is trial 0 with value: 0.5802196151770449.


Best trial: 1. Best value: 0.626587:   5%|▌         | 2/40 [00:11<03:17,  5.20s/it]

[I 2025-08-30 20:01:55,348] Trial 1 finished with value: 0.6265870801208628 and parameters: {'model_type': 'lgbm', 'lgbm_n_estimators': 756, 'lgbm_num_leaves': 58, 'lgbm_lr': 0.03157870399942338, 'lgbm_subsample': 0.7369003632320126, 'lgbm_colsample': 0.6251482979911165, 'lgbm_min_child_samples': 81, 'lgbm_alpha': 0.00034240966258748066, 'lgbm_lambda': 2.645081158570957e-07}. Best is trial 1 with value: 0.6265870801208628.


Best trial: 2. Best value: 0.650662:   8%|▊         | 3/40 [00:13<02:24,  3.91s/it]

...

[I 2025-08-30 20:04:48,487] Trial 37 finished with value: 0.6720757078534583 and parameters: {'model_type': 'xgb', 'xgb_n_estimators': 477, 'xgb_max_depth': 3, 'xgb_lr': 0.013436047448706563, 'xgb_subsample': 0.636385375546444, 'xgb_colsample': 0.7955327293112278, 'xgb_alpha': 0.03946320789636545, 'xgb_lambda': 1.3706944695549545e-08, 'xgb_min_child_weight': 8, 'xgb_gamma': 1.8888131891350917}. Best is trial 16 with value: 0.6729633709471536.


Best trial: 16. Best value: 0.672963:  98%|█████████▊| 39/40 [03:10<00:04,  4.86s/it]

[I 2025-08-30 20:04:54,799] Trial 38 finished with value: 0.6728431752906807 and parameters: {'model_type': 'xgb', 'xgb_n_estimators': 506, 'xgb_max_depth': 3, 'xgb_lr': 0.013809205742227134, 'xgb_subsample': 0.6275850136598596, 'xgb_colsample': 0.7741402979495506, 'xgb_alpha': 0.12712513365458816, 'xgb_lambda': 1.1751123641656253e-08, 'xgb_min_child_weight': 8, 'xgb_gamma': 1.733776316402445}. Best is trial 16 with value: 0.6729633709471536.


Best trial: 16. Best value: 0.672963: 100%|██████████| 40/40 [03:11<00:00,  4.78s/it]

[I 2025-08-30 20:04:55,558] Trial 39 pruned. 
Best value (PR-AUC): 0.6729633709471536
Best params: {'model_type': 'xgb', 'xgb_n_estimators': 201, 'xgb_max_depth': 4, 'xgb_lr': 0.026604318470071057, 'xgb_subsample': 0.6794910626490949, 'xgb_colsample': 0.8538589681187536, 'xgb_alpha': 0.00012548964513500356, 'xgb_lambda': 8.088844275488134, 'xgb_min_child_weight': 4, 'xgb_gamma': 3.9520539561675396}


In [8]:
# ⬅︎ Cell 8: Refit on train+valid and evaluate on test
best = study.best_params
best_type = best["model_type"]

if best_type == "xgb":
    model = XGBClassifier(
        n_estimators=best["xgb_n_estimators"],
        max_depth=best["xgb_max_depth"],
        learning_rate=best["xgb_lr"],
        subsample=best["xgb_subsample"],
        colsample_bytree=best["xgb_colsample"],
        reg_alpha=best["xgb_alpha"],
        reg_lambda=best["xgb_lambda"],
        min_child_weight=best["xgb_min_child_weight"],
        gamma=best["xgb_gamma"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective="binary:logistic",
        tree_method="hist",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight
    )
else:
    model = LGBMClassifier(
        n_estimators=best["lgbm_n_estimators"],
        num_leaves=best["lgbm_num_leaves"],
        learning_rate=best["lgbm_lr"],
        subsample=best["lgbm_subsample"],
        colsample_bytree=best["lgbm_colsample"],
        min_child_samples=best["lgbm_min_child_samples"],
        reg_alpha=best["lgbm_alpha"],
        reg_lambda=best["lgbm_lambda"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective="binary",
        metric="average_precision",
        verbose=-1,
        scale_pos_weight=scale_pos_weight
    )

pipe_best = make_pipeline(model)
pipe_best.fit(pd.concat([X_train, X_valid]), np.concatenate([y_train, y_valid]))

# Evaluate
metrics = evaluate(pipe_best, pd.concat([X_train, X_valid]), np.concatenate([y_train, y_valid]), X_test, y_test)
metrics


{'roc_auc_train': 0.875278993886619,
 'pr_auc_train': 0.7092686381097678,
 'roc_auc_valid': 0.8440469420699268,
 'pr_auc_valid': 0.6683111396512327}

In [9]:
# ⬅︎ Cell 9: Save artifacts
os.makedirs("../models", exist_ok=True)

joblib.dump(pipe_best, "../models/best_pipeline.pkl")

# שמירת מטא־דאטה כולל שמות פיצ'רים אחרי OneHot (יעיל להסברים וחשיבות פיצ'רים)
# שים לב: get_feature_names_out קיים בגרסאות sklearn חדשות
prep = pipe_best.named_steps["prep"]
feature_names = prep.get_feature_names_out()
meta = {
    "model_type": best_type,
    "best_params": best,
    "feature_names": feature_names.tolist()
}
with open("../models/metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved ../models/best_pipeline.pkl and ../models/metadata.json")


Saved ../models/best_pipeline.pkl and ../models/metadata.json


In [10]:
# ⬅︎ Cell 10: Feature importance (top 25)
import numpy as np
def get_importances(pipe):
    mdl = pipe.named_steps["model"]
    if hasattr(mdl, "feature_importances_"):
        imp = mdl.feature_importances_
    elif hasattr(mdl, "booster_") and hasattr(mdl.booster_, "feature_importances_"):
        imp = mdl.booster_.feature_importances_
    else:
        return None
    names = pipe.named_steps["prep"].get_feature_names_out()
    return pd.DataFrame({"feature": names, "importance": imp}).sort_values("importance", ascending=False)

fi = get_importances(pipe_best)
if fi is not None:
    display(fi.head(25))
    fi.to_csv("../models/feature_importance_top.csv", index=False)
else:
    print("No feature_importances_ available for this model.")


,feature,importance
6032,cat__Contract_Month-to-month,0.227361
11,num__contract_term,0.182746
6012,cat__InternetService_Fiber optic,0.080113
8,num__has_fiber,0.051039
6013,cat__InternetService_No,0.033348
12,num__monthly_x_term,0.031696
6014,cat__OnlineSecurity_No,0.028228
6023,cat__TechSupport_No,0.024719
6039,cat__PaymentMethod_Electronic check,0.023015
4,num__tenure_years,0.020648


In [11]:
# ⬅︎ Cell 11: Quick smoke test on a small sample
Xs = X_train.sample(200, random_state=RANDOM_STATE)
ys = y_train[np.isin(X_train.index, Xs.index)]

pipe_smoke = make_pipeline(LGBMClassifier(n_estimators=50, random_state=RANDOM_STATE))
pipe_smoke.fit(Xs, ys)
proba = pipe_smoke.predict_proba(Xs)[:,1]
print("Smoke PR-AUC:", average_precision_score(ys, proba))


Smoke PR-AUC: 0.9691152348567608
